# Move Map

A quick setup for mapping the places that your friends lived in over time and space.

By Nicolas Fröhlich, see [website post](https://www.nicolasfroehlich.de/projects/move-map/)

In [30]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd

In [32]:
df = pd.read_excel('movemap_data.xlsx')

In [38]:
# Convert date columns to timestamps
df["move_in_date"] = pd.to_datetime(df["move_in_date"], format="%Y/%m")
df["move_out_date"] = pd.to_datetime(df["move_out_date"], format="%Y/%m")

# Pick circle colors for each person_id
color_dict = {
    0: "black",
    1: "#1b9e77",
    2: "#d95f02",
    3: "#7570b3",
    4: "#e7298a",
    5: "#66a61e",
    6: "#e6ab02",
}

def monthly_range(start, end):
    """Return list of dates for the 1st of each month, start <= date <= end."""
    dates = []
    current = start.replace(day=1)
    last = end.replace(day=1)
    while current <= last:
        dates.append(current)
        # Increment to the next month
        y, m = current.year, current.month
        if m == 12:
            y += 1
            m = 1
        else:
            m += 1
        current = pd.to_datetime(f"{y}-{m:02d}-01")
    return dates



person_names = {
    0: "Everyone",
    1: "Anne",
    2: "Amari",
    3: "Bjørn",
    4: "Layla",
    5: "Diego",
    6: "Yi"
}

features = []
for _, row in df.iterrows():
    # Create a list of monthly timestamps from move_in_date to move_out_date
    times_list = monthly_range(row["move_in_date"], row["move_out_date"])
    
    # Assign color via 'iconstyle' so older Folium's TimestampedGeoJson can handle it
    marker_color = color_dict.get(row["person_id"], "blue")
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["lon"], row["lat"]]
        },
        "properties": {
            # The plugin uses 'times' to decide when to show/hide this circle
            "times": [t.isoformat() for t in times_list],
            "popup": (
                f"<b>{person_names.get(row['person_id'], 'Unknown')}</b> in "
                f"<b>{row['address_name']}</b><br>"
                f"<b>von:</b> {row['move_in_date'].strftime('%B %Y')}<br>"
                f"<b>bis:</b> {row['move_out_date'].strftime('%B %Y')}"
            ),
            # The critical part: "icon": "circle" + "iconstyle" for coloring
            "icon": "circle", # 🌿🍞
            "iconstyle": {
                "fillColor": marker_color,
                "fillOpacity": 0.8,
                "stroke": True,
                "color": marker_color,
                "radius": 11
            }
        }
    }
    features.append(feature)

person_data = {
    "type": "FeatureCollection",
    "features": features
}


# Create the map
m = folium.Map(location=[50, 10], zoom_start=4)

# Add Satellite basemap (Esri World Imagery) as another base layer
# folium.TileLayer(
#     tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr='Tiles &copy; Esri',
#     name='Satellite'
# ).add_to(m)

# Add the time-dimensional layer
TimestampedGeoJson(
    data=person_data,
    period="P1M",          # monthly steps
    duration='P1M',         # no trailing window
    add_last_point=False,   # remove circle after the last time
    auto_play=False,
    loop=True,
    max_speed=2,
    loop_button=True,
    date_options="YYYY/MM",
    time_slider_drag_update=True
).add_to(m)

title_html = '''
<div style="position: fixed; top: 10px; left: 25%; transform: translateX(-50%); 
     z-index: 9999; font-size: 24px; background-color: white; padding: 5px; border: 2px solid grey;">
  <b>Who lived when where? 🏡 🗺️ </b>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a legend in the bottom-right corner
legend_html = '''
 <div style="
 position: fixed;
 bottom: 30px;
 right: 30px;
 width: 100px;
 height: 165px;
 background-color: white;
 border:2px solid grey;
 z-index:9999;
 font-size:14px;
 padding: 10px;">
 <i style="color: black;">●</i> Everyone<br>
 <i style="color: #1b9e77;">●</i> Anne<br>
 <i style="color: #d95f02;">●</i> Amari<br>
 <i style="color: #7570b3;">●</i> Bjørn<br>
 <i style="color: #e7298a;">●</i> Layla<br>
 <i style="color: #66a61e;">●</i> Diego<br>
 <i style="color: #e6ab02;">●</i> Yi<br>
 </div>
 '''

m.get_root().html.add_child(folium.Element(legend_html))


# Add layer control to toggle between base layers
folium.LayerControl().add_to(m)

m


In [40]:
m.save('move_map.html')